## 목차
### 1.고객정보 전처리
### 2.고객 ID 기준 집계
- 네이버 스토어 고객 소비 패턴 파악

### ISSUE: 2024 1월 데이터 추가, 2023 1월 데이터 삭제
- 완전한 1년치 데이터 정리
- length 6531 > 7703

In [ ]:
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/낭만부부/analysis/final_nmbb.csv")

In [ ]:
df_add = pd.read_excel("/content/drive/MyDrive/낭만부부/analysis/sales_add.xlsx")

In [ ]:
df = pd.concat([df, df_add])
df.shape

(7117, 48)

In [ ]:
df_new = df[['결제일', '주문번호', '구매자명', '구매자ID', '상품번호', '상품명', '판매자 부담 할인액',
             '상품별 총 주문금액', '배송비 합계', '정산예정금액']]

### 결과: 10% 오분류 방지
- (전처리 전) 고객ID : 총 2552개
- (전처리 후) 고객ID : 총 2312개
- 240건 오분류 방지

In [ ]:
# 총 고객수: 2312명
df_new.groupby(['구매자명','구매자ID'])['주문번호'].unique()

구매자명  구매자ID       
l**   el********                                     [2023101314836451]
간**   db*******                                      [2023040913297111]
강**   12*****                                        [2023030711866051]
      31**********                 [2023091365534261, 2023083033061781]
      c0*****                                        [2023051741165131]
                                            ...                        
황성애   kj****          [2024011754617491, 2023122665513921, 202401238...
황연서   o0*****                                        [2024010183228641]
황지수   po*******                                      [2024020356268461]
황지현   hj****                                         [2023120565632161]
후**   wh*****                                        [2023072730655311]
Name: 주문번호, Length: 2552, dtype: object

In [ ]:
df_new['고객ID'] = df_new.groupby(['구매자명', '구매자ID']).ngroup()
df_new.head()

,결제일,주문번호,구매자명,구매자ID,상품번호,상품명,판매자 부담 할인액,상품별 총 주문금액,배송비 합계,정산예정금액,고객ID
0,2023-04-21 17:59:15,2023042110123081,송**,ha*********,6911507810,낭만부부 치즈몽땅 떡볶이 540g (치즈떡+만두+소스 밀키트),3560,14240,3000,13810,1143
1,2023-04-22 09:29:57,2023042218053241,박**,wi****,5892185888,낭만부부 꼬마가래떡 235g,0,5800,0,5250,905
2,2023-04-22 09:29:57,2023042218053241,박**,wi****,4788778377,낭만부부 햅쌀 떡국떡 1kg,0,15000,0,14547,905
3,2023-04-21 17:59:15,2023042110123081,송**,ha*********,6911541023,구워먹는 치즈바 530g (오리지널/단호박/흑임자),1780,7120,3000,6905,1143
4,2023-04-22 04:42:40,2023042216489621,문**,mh*****,4788783431,낭만부부 모짜렐라 치즈 가래떡 500g,9780,25820,0,27700,734


In [ ]:
df_new['고객ID'].nunique()

2552

### 문제점
- 총 1704개의 데이터가 구매자 비식별화 처리되지 않음.
- 대다수의 구매자명 컬럼은 강** 식으로 마스킹 처리 되어있지만, 최근구매자들은 비식별처리되지 않음
  - 예시
  - 강** (비식별처리 o)
  - 강유경 (비식별처리x)

In [ ]:
# 총 1704개의 미처리 건
df_new[~df_new['구매자명'].str.contains('\*\*')].head()

,결제일,주문번호,구매자명,구매자ID,상품번호,상품명,판매자 부담 할인액,상품별 총 주문금액,배송비 합계,정산예정금액,고객ID
4932,2024-01-18 13:36:11,2024011878278531,이철우,ch*******,9594976775,단백한 현미 단백질 다이어트 어린이 간식 야식 밀키트 떡볶이 540g,0,14900,3500,14450,1853
4933,2024-01-18 13:36:11,2024011878278531,이철우,ch*******,9511519851,치즈몽땅 떡볶이 540g 낭만부부 어린이 간식 야식 밀키트,0,9500,3500,9213,1853
4934,2024-01-18 12:15:17,2024011876449841,허윤정,yo********,9594976775,단백한 현미 단백질 다이어트 어린이 간식 야식 밀키트 떡볶이 540g,0,14900,3500,14450,2491
4935,2024-01-18 12:15:17,2024011876449841,허윤정,yo********,9511519851,치즈몽땅 떡볶이 540g 낭만부부 어린이 간식 야식 밀키트,0,9500,3500,9213,2491
4936,2024-01-18 11:32:10,2024011875451211,박다정,aj*****,9511519851,치즈몽땅 떡볶이 540g 낭만부부 어린이 간식 야식 밀키트,0,9500,3500,9213,923


In [ ]:
len(df_new[~df_new['구매자명'].str.contains('\*\*')])

1704

In [ ]:
df_name = df_new.copy()

## 집계를 위한 고객 정보 전처리
### `구매자명` 마스킹 처리
- 전처리 목적: 기간에 따라 일부 고객명은 비식별화 처리가 되어있지 않아 동일한 고객이더라도 다르게 집계됨.  
- 전처리 방법: 성을 제외한 이름을 "**"로 마스킹 처리
- 이후 비식별화된 **구매자명**과 **구매자ID** 를 Groupby로 묶어 `고객ID` 부여

In [ ]:
# '구매자명'에 마스킹처리
def mask_name(name):
    if len(name) == 2:
        return name[0] + '*'
    elif len(name) == 3:
        return name[0] + '**'
    else:
        return name

df_name['구매자명'] = df_name['구매자명'].apply(mask_name)

In [ ]:
df_name.groupby(['구매자명', '구매자ID'])['주문번호'].nunique()

구매자명  구매자ID       
l**   el********      1
간**   db*******       1
강**   12*****         1
      31**********    2
      al******        1
                     ..
황**   po*******       1
      se******        1
      si*******       1
      sy*******       1
후**   wh*****         1
Name: 주문번호, Length: 2312, dtype: int64

In [ ]:
# 고객ID 부여
df_name['고객ID'] = df_name.groupby(['구매자명', '구매자ID']).ngroup()
df_name.head()

,결제일,주문번호,구매자명,구매자ID,상품번호,상품명,판매자 부담 할인액,상품별 총 주문금액,배송비 합계,정산예정금액,고객ID
0,2023-04-21 17:59:15,2023042110123081,송**,ha*********,6911507810,낭만부부 치즈몽땅 떡볶이 540g (치즈떡+만두+소스 밀키트),3560,14240,3000,13810,1013
1,2023-04-22 09:29:57,2023042218053241,박**,wi****,5892185888,낭만부부 꼬마가래떡 235g,0,5800,0,5250,849
2,2023-04-22 09:29:57,2023042218053241,박**,wi****,4788778377,낭만부부 햅쌀 떡국떡 1kg,0,15000,0,14547,849
3,2023-04-21 17:59:15,2023042110123081,송**,ha*********,6911541023,구워먹는 치즈바 530g (오리지널/단호박/흑임자),1780,7120,3000,6905,1013
4,2023-04-22 04:42:40,2023042216489621,문**,mh*****,4788783431,낭만부부 모짜렐라 치즈 가래떡 500g,9780,25820,0,27700,645


In [ ]:
#전체고객
df_name['고객ID'].nunique()

2312

### 소비 패턴 확인
- 고객ID별 상품 총주문금액, 첫 결제일 컬럼 생성

### 구매주기 컬럼 추가
- (마지막 주문일 - 첫주문일) / 고객별 총 주문수
- RFM 기준으로 참고

In [ ]:
cust_order = df_name.groupby(['고객ID', '주문번호']).agg({'상품별 총 주문금액': 'sum',
                                                    '결제일':'first'}).reset_index()
cust_order.head()

,고객ID,주문번호,상품별 총 주문금액,결제일
0,0,2023101314836451,39900,2023-10-13 13:24:48
1,1,2023040913297111,16020,2023-04-09 23:51:43
2,2,2023030711866051,41400,2023-03-07 23:21:34
3,3,2023083033061781,11900,2023-08-30 17:50:28
4,3,2023091365534261,12600,2023-09-13 15:24:33


In [ ]:
cust_order['결제월'] = pd.to_datetime(cust_order['결제일']).dt.month

In [ ]:
#count -> nunique 로 해야 개별 상품 주문건이 아닌 1회 주문건으로 집계
df_name.groupby('고객ID')['주문번호'].nunique().max()

10

In [ ]:
df_name['결제일'] = pd.to_datetime(df_name['결제일'])

In [ ]:
result_df = df_name.groupby(['고객ID']).agg({
    '상품별 총 주문금액': 'sum',
    '결제일': ['min', 'max']}).reset_index()
    # 각 주문에 대한 최소 및 최대 주문일을 가져옴

In [ ]:
result_df.head()

고객ID 상품별 총 주문금액                 결제일                    
              sum                 min                 max
0    0      39900 2023-10-13 13:24:48 2023-10-13 13:24:48
1    1      16020 2023-04-09 23:51:43 2023-04-09 23:51:43
2    2      41400 2023-03-07 23:21:34 2023-03-07 23:21:34
3    3      24500 2023-08-30 17:50:28 2023-09-13 15:24:33
4    4       7600 2024-01-13 20:09:39 2024-01-13 20:09:39

In [ ]:
order_cnt = df_name.groupby('고객ID')['주문번호'].nunique().reset_index()
order_cnt.columns = ['고객ID', '고객별 총 주문수']

In [ ]:
result_df = pd.merge(result_df, order_count_by_customer, on='고객ID', how='left')

In [ ]:
result_df.head()

,고객ID,"(고객ID, )","(상품별 총 주문금액, sum)","(결제일, min)","(결제일, max)",고객별 총 주문수
0,0,0,39900,2023-10-13 13:24:48,2023-10-13 13:24:48,1
1,1,1,16020,2023-04-09 23:51:43,2023-04-09 23:51:43,1
2,2,2,41400,2023-03-07 23:21:34,2023-03-07 23:21:34,1
3,3,3,24500,2023-08-30 17:50:28,2023-09-13 15:24:33,2
4,4,4,7600,2024-01-13 20:09:39,2024-01-13 20:09:39,1


In [ ]:
result_df['첫구매'] = pd.to_datetime(result_df[('결제일', 'min')])
result_df['마지막구매'] = pd.to_datetime(result_df[('결제일', 'max')])

In [ ]:
result_df = result_df[[ '고객ID', ('상품별 총 주문금액', 'sum'),'고객별 총 주문수','첫구매','마지막구매']]

In [ ]:
result_df['첫구매'] = result_df['첫구매'].dt.date
result_df['마지막구매'] = result_df['마지막구매'].dt.date

In [ ]:
#재구매자: 566
result_df[result_df['고객별 총 주문수'] > 1]

,고객ID,"(상품별 총 주문금액, sum)",고객별 총 주문수,첫구매,마지막구매
3,3,24500,2,2023-08-30,2023-09-13
9,9,65280,3,2023-04-16,2023-11-16
12,12,236800,10,2023-06-11,2023-12-07
14,14,111700,3,2023-09-19,2024-02-01
20,20,88450,2,2023-06-02,2024-01-23
...,...,...,...,...,...
2285,2285,432200,2,2023-09-04,2023-10-24
2291,2291,47700,2,2023-06-20,2023-06-25
2294,2294,86200,3,2023-02-10,2023-10-20
2295,2295,70000,4,2023-06-23,2023-12-05


### 객단가 파악하기
- 전체고객
- 재구매 고객 (2회 이상 구매)

In [ ]:
result_df.rename(columns = {('상품별 총 주문금액', 'sum'):'총주문액'}, inplace = True)

In [ ]:
result_df['1회평균_주문액'] = result_df['총주문액'] / result_df['고객별 총 주문수']

In [ ]:
# 재구매자만 고려했을때 1회 주문시 주문액 = 30,000원
monetary_2 = result_df[result_df['고객별 총 주문수'] > 1]
monetary_2.describe()

,고객ID,총주문액,고객별 총 주문수,1회평균_주문액
count,566.000000,566.000000,566.000000,566.000000
mean,1084.992933,83400.141343,2.726148,30349.446800
std,678.943224,52464.111698,1.246981,12712.148122
min,3.000000,17800.000000,2.000000,8900.000000
25%,442.750000,53250.000000,2.000000,23727.500000
50%,1031.500000,68925.000000,2.000000,29941.666667
75%,1673.750000,97012.500000,3.000000,35691.666667
max,2298.000000,432200.000000,10.000000,216100.000000


In [ ]:
# 모든 구매자 고려
# 평균: 27,600원
# 중앙값: 27,600원
result_df.describe()

,고객ID,총주문액,고객별 총 주문수,1회평균_주문액
count,2312.000000,2312.000000,2312.000000,2312.000000
mean,1155.500000,40590.640138,1.422578,27603.316128
std,667.561233,37876.252065,0.965018,14466.574469
min,0.000000,3600.000000,1.000000,3600.000000
25%,577.750000,18400.000000,1.000000,17800.000000
50%,1155.500000,31100.000000,1.000000,27600.000000
75%,1733.250000,45150.000000,1.000000,34800.000000
max,2311.000000,432200.000000,10.000000,267000.000000


In [ ]:
# 1회 구매자 only: 26,700원
monetary_1 = result_df[result_df['고객별 총 주문수'] == 1]
monetary_1.describe()

,고객ID,총주문액,고객별 총 주문수,1회평균_주문액
count,1746.000000,1746.000000,1746.0,1746.000000
mean,1178.356243,26713.104238,1.0,26713.104238
std,662.416763,14886.291299,0.0,14886.291299
min,0.000000,3600.000000,1.0,3600.000000
25%,626.250000,17255.000000,1.0,17255.000000
50%,1190.500000,26700.000000,1.0,26700.000000
75%,1747.750000,34430.000000,1.0,34430.000000
max,2311.000000,267000.000000,1.0,267000.000000
